In [26]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [27]:
df = pd.read_csv("data/Surgical-deepnet.csv")
df

,bmi,Age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,baseline_osteoart,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
0,19.31,59.2,1,1,0,0,0,0,0,0,...,-0.57,3,0,7.63,6,1,0,-0.43,1,0
1,18.73,59.1,0,0,0,0,0,0,0,0,...,0.21,0,0,12.93,0,1,0,-0.41,1,0
2,21.85,59.0,0,0,0,0,0,0,0,0,...,0.00,2,0,7.68,5,3,0,0.08,1,0
3,18.49,59.0,1,0,1,0,0,1,1,0,...,-0.65,2,1,7.58,4,3,0,-0.32,1,0
4,19.70,59.0,1,0,0,0,0,0,0,0,...,0.00,0,0,7.88,11,0,0,0.00,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14630,18.79,14.1,1,0,1,0,0,0,0,0,...,-0.54,1,0,7.78,0,1,0,-0.16,1,1
14631,19.65,12.6,0,0,0,0,0,0,0,0,...,-1.42,4,0,8.40,6,1,0,-0.77,1,1
14632,14.84,12.6,1,0,0,0,0,0,0,0,...,0.65,0,0,13.25,3,3,0,0.99,1,1
14633,17.75,8.9,0,0,1,0,0,0,1,0,...,-0.50,0,1,8.30,5,0,0,0.17,1,1


In [28]:
df.isnull().sum()

bmi                    0
Age                    0
asa_status             0
baseline_cancer        0
baseline_charlson      0
baseline_cvd           0
baseline_dementia      0
baseline_diabetes      0
baseline_digestive     0
baseline_osteoart      0
baseline_psych         0
baseline_pulmonary     0
ahrq_ccs               0
ccsComplicationRate    0
ccsMort30Rate          0
complication_rsi       0
dow                    0
gender                 0
hour                   0
month                  0
moonphase              0
mort30                 0
mortality_rsi          0
race                   0
complication           0
dtype: int64

In [29]:
X = df.drop("complication",axis=1).copy().values
Y = df["complication"].copy().values

# data training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((11708, 24), (11708,), (2927, 24), (2927,))

In [30]:
class Perceptron:
    def __init__(self,learnig_rate,input_length):
        self.learnig_rate = learnig_rate
        self.weights = np.random.rand(input_length)
        self.bias = np.random.rand(1)

    def activation(self, x,  function):
        if function == "sigmoid":
            return 1 / (1 + np.exp(-x))
        elif function == "relu":
            return np.maximum(0, x)
        elif function == "tanh":
            return x
        else:
            raise Exception("Not supported activation function")
    def fit(self , X_train, Y_train ,epochs):
        for epoch in tqdm(range(epochs)):
            for x,y in zip(X_train,Y_train):
                #  forwarding
                y_pred = x @ self.weights + self.bias
                y_pred = self.activation(y_pred,"sigmoid")
                # back
                error = y- y_pred

                # updating
                self.weights += self.learnig_rate * error * x
                self.bias += self.learnig_rate * error 
        
    def predict(self , X_test):
        Y_pred = []
        for x_test in X_test:
            y_pred = self.forward(x_test)
            Y_pred.append(y_pred)
        return np.array(Y_pred)
    
    def calc_loss(self, X_test, Y_test, metric='mse'):
        y_pred = self.predict(X_test)
        if metric == 'mse':
            loss = np.mean((y_pred - Y_test) ** 2)
        elif metric == 'mae':
            loss = np.mean(np.abs(y_pred - Y_test))
        else:
            raise Exception('Not supported metric')
        return loss

    def calc_accuracy(self, X_test, Y_test):
        Y_pred = self.predict(X_test)
        Y_pred = np.where(Y_pred > 0.5,1,0)
        accuracy = np.sum(Y_pred == Y_test) / len(Y_test)
        return accuracy
        
    def evaluate(self,X_train,Y_train):
        loss = self.calc_loss(X_test, Y_test)
        accuracy = self.calc_accuracy(X_test, Y_test)
        return loss, accuracy  


In [31]:
model = Perceptron(learnig_rate=0.001,input_length=X_train.shape[1])
model.fit(X_train,Y_train, epochs=256)

100%|██████████| 256/256 [01:00<00:00,  4.21it/s]


In [32]:
model.evaluate(X_test,Y_test)

AttributeError: 'Perceptron' object has no attribute 'forward'